In [1]:
import pandas as pd
from darts import TimeSeries
from darts.models import ExponentialSmoothing
import matplotlib.pyplot as plt

In [2]:
path = '../../Data/Kaggle/StoreSales/processed_train_v2.csv'
df = pd.read_csv(path, index_col='date', parse_dates=['date'])

In [3]:
df.head()

,id,store_nbr,family,sales,onpromotion,typeholiday,dcoilwtico,city,state,typestores,cluster,day_of_week,day,month,year
date,,,,,,,,,,,,,,,
2013-01-01,0,1,0,0.0,0,3,93.14,18,12,3,13,2,1,1,2013
2013-01-01,1,1,1,0.0,0,3,93.14,18,12,3,13,2,1,1,2013
2013-01-01,2,1,2,0.0,0,3,93.14,18,12,3,13,2,1,1,2013
2013-01-01,3,1,3,0.0,0,3,93.14,18,12,3,13,2,1,1,2013
2013-01-01,4,1,4,0.0,0,3,93.14,18,12,3,13,2,1,1,2013


In [4]:
df_sales = df.loc[(df['store_nbr'].isin([1])) & (df['family'].isin([3])), ['sales']]

In [5]:
df_dcoilwtico = df.loc[(df['store_nbr'].isin([1])) & (df['family'].isin([3])), ['dcoilwtico']]

In [6]:
df_sales = df_sales.astype('float32')

In [7]:
df_sales.head()

,sales
date,
2013-01-01,0.0
2013-01-02,1091.0
2013-01-03,919.0
2013-01-04,953.0
2013-01-05,1160.0


In [8]:
df_dcoilwtico.head()

,dcoilwtico
date,
2013-01-01,93.14000
2013-01-02,93.14000
2013-01-03,92.97000
2013-01-04,93.12000
2013-01-05,93.12009


In [9]:
series_sales = TimeSeries.from_dataframe(df_sales, freq='D')

In [10]:
series_dcoilwtico = TimeSeries.from_dataframe(df_dcoilwtico, freq='D')

In [11]:
split_index = int(0.8 * len(df_sales))

In [12]:
train_sales, val_sales = series_sales[:split_index], series_sales[split_index:]

In [13]:
train_dcoilwtico, val_dcoilwtico = series_dcoilwtico[:split_index], series_dcoilwtico[split_index:]

In [63]:
fc = len(df_sales) - split_index

In [64]:
target = train_sales

future_cov = series_dcoilwtico[split_index:]

In [65]:
target

<TimeSeries (DataArray) (date: 1347, component: 1, sample: 1)>
array([[[   0.]],

       [[1091.]],

       [[ 919.]],

       ...,

       [[2391.]],

       [[2060.]],

       [[1946.]]], dtype=float32)
Coordinates:
  * date       (date) datetime64[ns] 2013-01-01 2013-01-02 ... 2016-09-08
  * component  (component) object 'sales'
Dimensions without coordinates: sample
Attributes:
    static_covariates:  None
    hierarchy:          None

In [66]:
from darts.models import XGBModel

model = XGBModel(
    lags_future_covariates=(0, fc)
)

In [67]:
model.fit(target, future_covariates=future_cov)

ValueError: Specified series do not share any common times for which features can be created.


ValueError: Specified series do not share any common times for which features can be created.

In [58]:
prediction = model.predict(1)
prediction

ValueError: The corresponding future_covariate of the series at index 0 isn't sufficiently long. Given horizon `n=1`, `min(lags_future_covariates)=0`, `max(lags_future_covariates)=1446` and `output_chunk_length=1`, the future_covariate has to range from 2016-09-09 00:00:00 until 2020-08-25 00:00:00 (inclusive), but it ranges only from 2013-01-01 00:00:00 until 2016-12-17 00:00:00.


ValueError: The corresponding future_covariate of the series at index 0 isn't sufficiently long. Given horizon `n=1`, `min(lags_future_covariates)=0`, `max(lags_future_covariates)=1446` and `output_chunk_length=1`, the future_covariate has to range from 2016-09-09 00:00:00 until 2020-08-25 00:00:00 (inclusive), but it ranges only from 2013-01-01 00:00:00 until 2016-12-17 00:00:00.

In [ ]:
series.plot()
prediction.plot(label="forecast", low_quantile=0.05, high_quantile=0.95)
plt.legend()